<a href="https://colab.research.google.com/github/SandeeeeeeeeepDey/Kaggle_noob_to_pro/blob/main/Titanic/titanic_kaggle_tfdf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Kaggle_Competition Jump start



##The Challenge
The sinking of the Titanic is one of the most infamous shipwrecks in history.

On April 15, 1912, during her maiden voyage, the widely considered “unsinkable” RMS Titanic sank after colliding with an iceberg. Unfortunately, there weren’t enough lifeboats for everyone onboard, resulting in the death of 1502 out of 2224 passengers and crew.

While there was some element of luck involved in surviving, it seems some groups of people were more likely to survive than others.

In this challenge, we ask you to build a predictive model that answers the question: “what sorts of people were more likely to survive?” using passenger data (ie name, age, gender, socio-economic class, etc).

##Import and Load

In [2]:
!pip install tensorflow_decision_forests

In [3]:
import numpy as np
import pandas as pd
import os

import tensorflow as tf
import tensorflow_decision_forests as tfdf

In [4]:
train_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Titanic_Kaggle/train.csv")
test_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Titanic_Kaggle/train.csv")
train_df.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


##Dataset Preprocessing/Preparation

Make one preprocess function to use on both training set and test set.

In [5]:
def preprocess(df):
  df = df.copy()

  def normalize_name(x):
    return " ".join([v.strip(""",()[]."'""") for v in x.split(" ")])

  def ticket_number(x):
    return x.split(" ")[-1]

  def ticket_item(x):
    items = x.split(" ")
    if len(items)<2:
      return "NONE"
    return "_".join(items[0:-1])

  df["Name"] = df["Name"].apply(normalize_name)
  df["Ticket_number"] = df["Ticket"].apply(ticket_number)
  df["Ticket_item"] = df["Ticket"].apply(ticket_item)
  return df

preprocessed_train_df = preprocess(train_df)
preprocessed_test_df = preprocess(test_df)

In [6]:
preprocessed_train_df.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Ticket_number,Ticket_item
0,1,0,3,Braund Mr Owen Harris,male,22.0,1,0,A/5 21171,7.2500,NaN,S,21171,A/5
1,2,1,1,Cumings Mrs John Bradley Florence Briggs Thayer,female,38.0,1,0,PC 17599,71.2833,C85,C,17599,PC
2,3,1,3,Heikkinen Miss Laina,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,3101282,STON/O2.
3,4,1,1,Futrelle Mrs Jacques Heath Lily May Peel,female,35.0,1,0,113803,53.1000,C123,S,113803,NONE
4,5,0,3,Allen Mr William Henry,male,35.0,0,0,373450,8.0500,NaN,S,373450,NONE
5,6,0,3,Moran Mr James,male,NaN,0,0,330877,8.4583,NaN,Q,330877,NONE
6,7,0,1,McCarthy Mr Timothy J,male,54.0,0,0,17463,51.8625,E46,S,17463,NONE
7,8,0,3,Palsson Master Gosta Leonard,male,2.0,3,1,349909,21.0750,NaN,S,349909,NONE
8,9,1,3,Johnson Mrs Oscar W Elisabeth Vilhelmina Berg,female,27.0,0,2,347742,11.1333,NaN,S,347742,NONE
9,10,1,2,Nasser Mrs Nicholas Adele Achem,female,14.0,1,0,237736,30.0708,NaN,C,237736,NONE


Remove columns that will surely not be of any use as a feature.
Saves processing time and reduces dimension

In [7]:
input_features = list(preprocessed_train_df.columns)
input_features.remove("Survived")
input_features.remove("Ticket")
input_features.remove("PassengerId")

print(input_features)

['Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Cabin', 'Embarked', 'Ticket_number', 'Ticket_item']


In [8]:
preprocessed_train_df.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Ticket_number,Ticket_item
0,1,0,3,Braund Mr Owen Harris,male,22.0,1,0,A/5 21171,7.2500,NaN,S,21171,A/5
1,2,1,1,Cumings Mrs John Bradley Florence Briggs Thayer,female,38.0,1,0,PC 17599,71.2833,C85,C,17599,PC
2,3,1,3,Heikkinen Miss Laina,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,3101282,STON/O2.
3,4,1,1,Futrelle Mrs Jacques Heath Lily May Peel,female,35.0,1,0,113803,53.1000,C123,S,113803,NONE
4,5,0,3,Allen Mr William Henry,male,35.0,0,0,373450,8.0500,NaN,S,373450,NONE


## Pandas dataframe to Tensor Dataset


work with python, but at the end use Tensors

In [9]:
def tokenize_names(features, labels=None):
  """
  TFDF can natively only use tokens, so we convert names to tokens.
  It can however handle string type catagorical datas
  """
  features["Name"] = tf.strings.split(features["Name"])
  return features, labels

train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(preprocessed_train_df, label = "Survived").map(tokenize_names)
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(preprocessed_test_df).map(tokenize_names)


##Train Models

###Train with Default Params

In [10]:
model = tfdf.keras.GradientBoostedTreesModel(
    features = [tfdf.keras.FeatureUsage(name=n) for n in input_features],
    exclude_non_specified_features=True,
    random_seed = 1234
  )
model.fit(train_ds)

Use /tmp/tmpeimp8mee as temporary training directory
Reading training dataset...
Training dataset read in 0:00:05.191301. Found 891 examples.
Training model...
Model trained in 0:00:00.343907
Compiling model...
Model compiled.


In [11]:
self_evaluation = model.make_inspector().evaluation()

In [12]:
print(f"Acc:{self_evaluation.accuracy} Loss:{self_evaluation.loss}")

Acc:0.8260869383811951 Loss:0.8608942627906799


In [13]:
model.summary()

Model: "gradient_boosted_trees_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 1 (1.00 Byte)
Trainable params: 0 (0.00 Byte)
Non-trainable params: 1 (1.00 Byte)
_________________________________________________________________
Type: "GRADIENT_BOOSTED_TREES"
Task: CLASSIFICATION
Label: "__LABEL"

Input Features (11):
	Age
	Cabin
	Embarked
	Fare
	Name
	Parch
	Pclass
	Sex
	SibSp
	Ticket_item
	Ticket_number

No weights

Variable Importance: INV_MEAN_MIN_DEPTH:
    1.           "Sex"  0.459973 ################
    2.          "Name"  0.275098 #####
    3.        "Pclass"  0.245396 ###
    4.          "Fare"  0.236151 ###
    5.           "Age"  0.236094 ###
    6.   "Ticket_item"  0.206166 #
    7. "Ticket_number"  0.199365 #
    8.      "Embarked"  0.184753 
    9.         "SibSp"  0.181901 
   10.         "Parch"  0.177765 

Variable Importance: NUM_AS_ROOT:
    1.  "Sex" 23.000000 ###

###Improved Default Params

In [14]:
model = tfdf.keras.GradientBoostedTreesModel(
    verbose = 0,
    features = [tfdf.keras.FeatureUsage(name=n) for n in input_features],
    exclude_non_specified_features=True,
    min_examples = 1,
    categorical_algorithm="RANDOM",
    shrinkage=0.05,
    split_axis="SPARSE_OBLIQUE",
    sparse_oblique_normalization = "MIN_MAX",
    sparse_oblique_num_projections_exponent = 2.0,
    num_trees=2000,
    random_seed=1234
)

In [15]:
model.fit(train_ds)

In [16]:
self_evaluation = model.make_inspector().evaluation()
print(f"Acc:{self_evaluation.accuracy} Loss:{self_evaluation.loss}")

Acc:0.75 Loss:1.0462466478347778


In [17]:
model.summary()

Model: "gradient_boosted_trees_model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 1 (1.00 Byte)
Trainable params: 0 (0.00 Byte)
Non-trainable params: 1 (1.00 Byte)
_________________________________________________________________
Type: "GRADIENT_BOOSTED_TREES"
Task: CLASSIFICATION
Label: "__LABEL"

Input Features (11):
	Age
	Cabin
	Embarked
	Fare
	Name
	Parch
	Pclass
	Sex
	SibSp
	Ticket_item
	Ticket_number

No weights

Variable Importance: INV_MEAN_MIN_DEPTH:
    1.           "Sex"  0.805245 ################
    2.           "Age"  0.372369 #####
    3.          "Fare"  0.273026 ##
    4.          "Name"  0.187307 
    5.        "Pclass"  0.180805 
    6.   "Ticket_item"  0.178416 
    7. "Ticket_number"  0.178366 
    8.         "Parch"  0.177684 
    9.      "Embarked"  0.176070 
   10.         "SibSp"  0.172557 

Variable Importance: NUM_AS_ROOT:
    1.  "Sex" 34.000000 ##########

####Make predictions

In [18]:
def prediction_to_kaggle_format(model, threshold=0.5):
  proba_survive = model.predict(test_ds, verbose=0)[:,0]
  return pd.DataFrame({
      "PassengerId": test_df["PassengerId"],
      "Survived": (proba_survive >= threshold).astype(int)
  })

def make_submission(kaggle_predictions):
  path = "/content/kaggle/submission.csv"
  kaggle_predictions.to_csv(path, index = False)

kaggle_predictions = prediction_to_kaggle_format(model)
make_submission(kaggle_predictions)
!head /content/kaggle/submission.csv

PassengerId,Survived
1,0
2,1
3,1
4,1
5,0
6,0
7,0
8,0
9,1


###Model with hyperparameter tunning

Tuner argument configuration

In [19]:
tuner = tfdf.tuner.RandomSearch(num_trials=1000)

tuner.choice("min_examples", [1,5,7,10])

tuner.choice("categorical_algorithm", ["CART","RANDOM"])

local_search_space = tuner.choice("growing_strategy", ["LOCAL"])
local_search_space.choice("max_depth", [3,4,5,6,7,8])

global_search_space = tuner.choice("growing_strategy",["BEST_FIRST_GLOBAL"], merge=True)
global_search_space.choice("max_num_nodes", [16,32,64,128,256])

tuner.choice("shrinkage", [0.02,0.05,0.1,0.15])
tuner.choice("num_candidate_attributes_ratio", [0.2, 0.5, 0.9, 1.0])

tuner.choice("split_axis", ["AXIS_ALIGNED"])
oblique_space = tuner.choice("split_axis", ["SPARSE_OBLIQUE"], merge=True)
oblique_space.choice("sparse_oblique_normalization",["NONE", "STANDARD_DEVIATION","MIN_MAX"])
oblique_space.choice("sparse_oblique_weights", ["BINARY", "CONTINUOUS"])
oblique_space.choice("sparse_oblique_num_projections_exponent", [1.0,1.5,2.0])

Make model based on the tuner arg

In [20]:
tuned_model = tfdf.keras.GradientBoostedTreesModel(tuner=tuner)
tuned_model.fit(train_ds, verbose=1)

Use /tmp/tmpuxfboxdj as temporary training directory
Reading training dataset...
Training dataset read in 0:00:00.362302. Found 891 examples.
Training model...
Model trained in 0:07:05.854733
Compiling model...
Model compiled.


In [21]:
tunes_self_evaluation = tuned_model.make_inspector().evaluation()
print(f"Acc:{tunes_self_evaluation.accuracy} Loss:{tunes_self_evaluation.loss}")

Acc:0.9041095972061157 Loss:0.6523688435554504


#####Ensemble multiple runs to get the agv output.(no luck)

In [ ]:
predictions = None
num_predictions = 0

for i in range(100):
  print(f"i:{i}")

  model = tfdf.keras.GradientBoostedTreesModel(tuner=tuner)
  model.fit(train_ds,verbose=1)
  sub_predictions = model.predict(train_ds, verbose=1)[:,0]
  if predictions is None:
    predictions = sub_predictions
  else:
    predictions += sub_predictions
  num_predictions += 1

predictions /= num_predictions

i:0
Use /tmp/tmpjc5p1uqa as temporary training directory
Reading training dataset...
Training dataset read in 0:00:00.316879. Found 891 examples.
Training model...
Model trained in 0:07:10.170656
Compiling model...


Model compiled.


1/1 [==============================] - 0s 212ms/step
i:1
Use /tmp/tmpb_dw39se as temporary training directory
Reading training dataset...


Training dataset read in 0:00:00.540337. Found 891 examples.
Training model...
Model trained in 0:07:05.684040
Compiling model...
Model compiled.


1/1 [==============================] - 0s 208ms/step
i:2
Use /tmp/tmpkidn6nwr as temporary training directory
Reading training dataset...


Training dataset read in 0:00:00.535203. Found 891 examples.
Training model...
Model trained in 0:07:02.839302
Compiling model...
Model compiled.


1/1 [==============================] - 0s 123ms/step
i:3
Use /tmp/tmp4q5p8kxc as temporary training directory
Reading training dataset...
Training dataset read in 0:00:00.329061. Found 891 examples.
Training model...
Model trained in 0:07:03.444435
Compiling model...
Model compiled.
1/1 [==============================] - 0s 132ms/step
i:4
Use /tmp/tmp7twnqwub as temporary training directory
Reading training dataset...
Training dataset read in 0:00:00.364699. Found 891 examples.
Training model...
Model trained in 0:07:05.095445
Compiling model...
Model compiled.
1/1 [==============================] - 0s 212ms/step
i:5
Use /tmp/tmpnzax55hy as temporary training directory
Reading training dataset...
Training dataset read in 0:00:00.491666. Found 891 examples.
Training model...
Model trained in 0:07:02.875456
Compiling model...
Model compiled.
1/1 [==============================] - 0s 124ms/step
i:6
Use /tmp/tmpa1q8ix98 as temporary training directory
Reading training dataset...
Training d

In [1]:
kaggle_predictions = pd.Dataframe({
    "PassangerID": test_df["PassangerID"]
    "survived": (predictions >= 0.5).astype(int)
})

SyntaxError: invalid syntax. Perhaps you forgot a comma? (<ipython-input-1-190ad56c5e85>, line 2)

In [ ]:
make_submission(kaggle_predictions)
